Hamiltoniano:

$$ H = - \sum_{\langle i, j \rangle} J s_i s_j - \sum_{i}(H(t) + h_i)s_i $$

El campo aleatorio se saca de la distribución:

$$ P(h) =  \frac{1}{\sqrt{2 \pi} R} e^{-h^2/2R^2} $$

In [4]:
using PyPlot

INFO: Loading help data...


In [1]:
type MicroEstado
    σ::Array{Int,2}
    h::Array{Float64,2}
    #Vamos a suponer que todas las configuraciones son cuadradas
    L::Int
end

import Base.show 

show(io::IO, m::MicroEstado) = print(io, m.σ)

import PyPlot.imshow

imshow(m::MicroEstado) = imshow(m.σ, interpolation="none")d

show (generic function with 91 methods)

In [2]:
function edo_inicial(L::Int)
    σ = -ones(Int, (L,L))
    
    h = Array(Float64, (L,L))
    for i in 1:L^2
        # Temporal, hay que cambiar la distribución
        h[i] = randn()
    end
        
    MicroEstado(σ, h, L)
end

edo_inicial (generic function with 1 method)

In [53]:
function energia_espin(m::MicroEstado, i::Int, j::Int)
    σ = m.σ ; L = m.L
    
    #Tal vez deberíamos de cambiar el signo de la energía
    σ[i,j]*( σ[mod1(i-1, L),j] + σ[mod1(i+1, L),j] + σ[i,mod1(j-1, L)] + σ[i,mod1(j+1, L)] ) + 
    m.h[i,j]
end

energia_espin (generic function with 1 method)

In [5]:
function voltea_espin!(m::MicroEstado, i::Int, j::Int)
    if m.σ[i,j] == -1
        m.σ[i,j] *= -1
    end
end

voltea_espin! (generic function with 1 method)

In [6]:
function max_energia_abajo(m::MicroEstado,min_energia)
    energias = Array(Float64,(m.L,m.L))
    
    for i = 1:m.L, j = 1:m.L
        if m.σ[i,j] == -1
            energias[i,j] = energia_espin(m,i,j)
        else
            energias[i,j] = min_energia
        end
    end
    
    #Si cambiamos signo de la energía sería findmin
    f = findmax(energia) # Da el máximo y el ínidice lineal del máximo
    i = mod1(f[2], m.L)
    j = int(ceil(f[2]/m.L))
    
    f[1], i, j
end

max_energia_abajo (generic function with 1 method)

In [7]:
function espines_vecinos_abajo(m::MicroEstado,i::Int,j::Int)
    vecinos = [ (mod1(i-1,m.L),j), (mod1(i+1,m.L),j), (i,mod1(j-1,m.L)), (i,mod1(j+1,m.L)) ]
    vecinos_abajo = (Int,Int)[]
    
    for k in vecinos
        if m.σ[k...] == -1
            push!(vecinos_abajo, k)
        end
    end
    
    vecinos_abajo        
end

espines_vecinos_abajo (generic function with 1 method)

In [8]:
function avalancha(m::MicroEstado, i::Int, j::Int, H::Float64)    
    voltea_espin!(m,i,j)
    #puede haber un problema, si se escoje de entrada un espín que ya está volteado
    espines_volteados = [(i,j)]
    
    candidatos = espines_vecinos_abajo(m,i,j)
    
    while length(candidatos) > 0
        nuevos_candidatos = (Int,Int)[]
        
        for k in candidatos
            if -energia_espin(m,k...) > H 
                voltea_espin!(m,k...])
                push!(espines_volteados,k)
                push!(nuevos_candidatos, espines_vecinos_abajo(m,k...)
#                 nuevos_candidatos = vcat(nuevos_candidatos,espines_vecinos_abajo(m,k[1],k[2]))
            end
        end
        
        candidatos = nuevos_candidatos
    end
    
    m, espines_volteados
end

avalancha (generic function with 1 method)

In [84]:
T = edo_inicial(6)
energia_minima = minimum(T.h)-5
max_energia_abajo(T,energia_minima)

(6.37057875537757,6,3)

In [85]:
T.h

6x6 Array{Float64,2}:
 -1.66264   -0.706165    1.36966    0.389671  -0.825701   1.94185  
 -0.1798    -0.183372    0.302822   0.868958  -0.628087  -0.919909 
  0.327284  -1.75213    -0.867008  -1.95993    0.481889  -2.21498  
  0.422589   0.0284989   0.200689   0.404626  -1.45407   -0.0199838
  0.629627   0.368809    1.81832   -0.691898   1.24912   -0.776488 
 -0.442832   1.27428     2.37058    0.415274   0.342536  -0.648113 

In [86]:
while sum(T.σ) < T.L^2
#     figure(figsize=(4,4))
#     imshow(T)
    H,i,j = max_energia_abajo(T,energia_minima)
    @show H-4
    T, volteados = avalancha(T,i,j,H)
    @show volteados
    println(T)
    println()
end

H - 4 => 2.3705787553775703
volteados => [(6,3)]
[-1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 1 -1 -1 -1]

H - 4 => 1.9418499042339823
volteados => [(1,6)]
[-1 -1 -1 -1 -1 1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 1 -1 -1 -1]

H - 4 => 1.249117243780404
volteados => [(5,5)]
[-1 -1 -1 -1 -1 1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 1 -1
 -1 -1 1 -1 -1 -1]

H - 4 => 0.8689584532562735
volteados => [(2,4)]
[-1 -1 -1 -1 -1 1
 -1 -1 -1 1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 1 -1
 -1 -1 1 -1 -1 -1]

H - 4 => 0.6296271375564935
volteados => [(5,1)]
[-1 -1 -1 -1 -1 1
 -1 -1 -1 1 -1 -1
 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1
 1 -1 -1 -1 1 -1
 -1 -1 1 -1 -1 -1]

H - 4 => 0.48188909161392157
volteados => [(3,5)]
[-1 -1 -1 -1 -1 1
 -1 -1 -1 1 -1 -1
 -1 -1 -1 -1 1 -1
 -1 -1 -1 -1 -1 -1
 1 -1 -1 -1 1 -1
 -1 -1 1 -1 -1 -1]

H - 4 => 0.4046257495197647
vol